In [3]:
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

w, h = model_wh('0x0')
if w == 0 or h == 0:
    e = TfPoseEstimator(get_graph_path('cmu'), target_size=(432, 368))
else:
    e = TfPoseEstimator(get_graph_path('cmu'), target_size=(w, h))

image = common.read_imgfile('./images/p2.jpg', None, None)
image_h, image_w = image.shape[:2]
t = time.time()
humans = e.inference(image, resize_to_default=(w > 0 and h > 0), upsample_size=4.0)
elapsed = time.time() - t
log = []
for human in humans:
    centers = {}
    for i in range(common.CocoPart.Background.value):
        if i not in human.body_parts.keys():
            continue
        body_part = human.body_parts[i]
        center = (int(body_part.x * image_w + 0.5), int(body_part.y * image_h + 0.5))
        centers[i] = center
    log.append(centers)

print(len(humans))
img = TfPoseEstimator.draw_humans(image, humans, imgcopy=False)
cv2.imshow('frame', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[2019-10-07 12:14:18,298] [TfPoseEstimator] [INFO] loading graph from /home/yinghuit/Documents/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)
I1007 12:14:18.298104 140534574683968 estimator.py:310] loading graph from /home/yinghuit/Documents/tf-pose-estimation/models/graph/cmu/graph_opt.pb(default size=432x368)


2


In [5]:
log

[{0: (354, 84),
  1: (360, 130),
  2: (314, 140),
  3: (302, 212),
  4: (326, 244),
  5: (406, 122),
  6: (442, 192),
  7: (392, 248),
  8: (336, 250),
  9: (260, 280),
  10: (310, 400),
  11: (408, 254),
  12: (342, 288),
  13: (354, 408),
  14: (344, 76),
  15: (364, 76),
  16: (330, 88),
  17: (378, 88)},
 {0: (136, 78),
  1: (138, 120),
  2: (94, 130),
  3: (84, 196),
  4: (106, 224),
  5: (182, 112),
  6: (216, 178),
  7: (170, 228),
  8: (118, 232),
  9: (44, 260),
  10: (90, 374),
  11: (184, 234),
  12: (120, 266),
  13: (134, 386),
  14: (126, 70),
  15: (146, 70),
  16: (108, 74),
  17: (154, 76)}]

## To Draw Human Pose Centers and Lines

In [3]:
# Testing

image_h = 1080
image_w = 1920
jsonstring = '[{16: (1690, 790), 2: (1684, 802), 3: (1678, 812), 4: (1696, 796)}, {1: (1902, 320), 8: (1904, 340), 9: (1900, 360), 10: (1896, 374), 11: (1904, 340), 12: (1902, 360), 13: (1896, 372)}]'

In [ ]:
import cv2
import numpy as np

from tf_pose import common
from tf_pose.common import CocoPart

img = np.zeros([image_h,image_w,3],dtype=np.uint8)
img.fill(0)
log = eval(jsonstring)
print(log)
print(len(log))
for obj in log:
    for k, v in obj.items():
        cv2.circle(img, v, 3, common.CocoColors[k], thickness=3, lineType=8, shift=0)
    for pair_order, pair in enumerate(common.CocoPairsRender):
        if pair[0] not in obj.keys() or pair[1] not in obj.keys():
            continue
        cv2.line(img, obj[pair[0]], obj[pair[1]], common.CocoColors[pair_order], 3)
cv2.imshow('draw', img)
cv2.waitKey(0)
cv2.destroyAllWindows()

[{16: (1690, 790), 2: (1684, 802), 3: (1678, 812), 4: (1696, 796)}, {1: (1902, 320), 8: (1904, 340), 9: (1900, 360), 10: (1896, 374), 11: (1904, 340), 12: (1902, 360), 13: (1896, 372)}]
2
